In [1]:
import pandas as pd
import numpy as np
import random
from ortools.sat.python import cp_model

In [2]:
def doub_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    nova_linha_3 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2, nova_linha_3])

def tri_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S').strftime('%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    nova_linha_3 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    nova_linha_3['Horário'] = (hora_inicial + pd.Timedelta(hours=2)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2, nova_linha_3])

def desaninhar_dias(df):
    df['Dias da Semana'] = df['Dias da Semana'].str.split(',')
    df = df.explode('Dias da Semana')
    return df

# Aplicar a função para todas as linhas triplas
def apply_func(df,func):
    linhas_triplicadas = df.apply(func, axis=1)
    return pd.concat(linhas_triplicadas.tolist(), ignore_index=True)


In [45]:
aulas = pd.read_excel('/home/cayena/Downloads/planilha_teste_thefamily.xlsx')

In [23]:
tri = aulas[aulas['Dias da Semana'] == 'Saturday - Triple'].copy()
doub = aulas[aulas['Dias da Semana'].str.contains('DOUBLE')].copy()
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()
aulas_simples['Horário'] = pd.to_datetime(aulas_simples['Horário'],format='%H:%M:%S').dt.strftime('%H:%M:%S')

In [6]:
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

In [22]:
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','').str.replace('  ',',')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

In [5]:
aulas[aulas['Dias da Semana'] == 'Saturday - Triple']

,Grupo,Horário,Unidade,Dias da Semana
6,GIBRALTAR PRESENCIAL,08:00:00,Vicentina,Saturday - Triple
13,BRISBANE PRESENCIAL,08:00:00,Satélite,Saturday - Triple
16,NORTHAMPTOM PRESENCIAL,08:00:00,Satélite,Saturday - Triple
18,SOUTHAMPTON PRESENCIAL,08:00:00,Satélite,Saturday - Triple
19,BARCELONA PRESENCIAL,08:00:00,Satélite,Saturday - Triple
21,CHARLESTON PRESENCIAL,08:00:00,Satélite,Saturday - Triple
22,NEBRASKA PRESENCIAL,08:00:00,Satélite,Saturday - Triple
57,POLAND PRESENCIAL,08:00:00,Vicentina,Saturday - Triple


In [4]:
aulas['Dias da Semana'].value_counts()

2ª , 3ª , 4ª , 5ª      27
Saturday - Triple       8
3ª                      6
2ª , 4ª , 5ª            6
2ª , 4ª                 5
2ª , 3ª , 5ª            5
2ª                      4
5ª                      3
3ª , 5ª                 3
3ª , 4ª , 5ª            2
6ª                      2
2ª  , 4ª                1
2ª DOUBLE               1
4ª                      1
2ª DOUBLE 4ª DOUBLE     1
4ª                      1
Name: Dias da Semana, dtype: int64

In [6]:
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.split(',')

In [8]:
aulas.head(20)

,Grupo,Horário,Unidade,Dias da Semana
0,MBA DAVI BRAGA,14:00:00,Vicentina,[2ª]
1,MBA LIGA DOS CAMPEÕES,18:30:00,Vicentina,[5ª]
2,MBA IDEIAS EM AÇÃO,19:00:00,Vicentina,[2ª]
3,SAVANAH PRESENCIAL,07:00:00,Vicentina,"[2ª , 3ª , 4ª , 5ª]"
4,CONVERSATION 14 PRESENCIAL,08:00:00,Vicentina,[3ª]
5,FERNANDO CAJE - PRESENCIAL,08:00:00,Vicentina,"[2ª , 4ª]"
6,GIBRALTAR PRESENCIAL,08:00:00,Vicentina,[Saturday - Triple]
7,KNOXVILLE PRESENCIAL,08:00:00,Vicentina,"[2ª , 3ª , 4ª , 5ª]"
8,GREAT BRITAIN PRESENCIAL,09:00:00,Vicentina,"[2ª , 3ª , 4ª , 5ª]"
9,GARLAND PRESENCIAL,18:00:00,Vicentina,"[2ª , 3ª , 4ª , 5ª]"


In [4]:
aulas['segunda'] = aulas['Dias da Semana'].str.contains('2ª').astype(int)
aulas['terça'] = aulas['Dias da Semana'].str.contains('3ª').astype(int)
aulas['quarta'] = aulas['Dias da Semana'].str.contains('4ª').astype(int)
aulas['quinta'] = aulas['Dias da Semana'].str.contains('5ª').astype(int)

In [5]:
df_horarios = aulas['Horário'].str.get_dummies()
df_unidade = aulas['Unidade'].str.get_dummies()

In [6]:
aulas_geral = pd.concat([aulas, df_horarios,df_unidade], axis=1)

In [7]:
grupo_semana = aulas_geral[['Grupo', 'segunda', 'terça', 'quarta', 'quinta']].copy()
grupo_horario = aulas_geral.drop(columns=[ 'Horário', 'Unidade', 'Dias da Semana','segunda', 'terça', 'quarta', 'quinta', 'Jardim', 'Satélite', 'Vicentina']).copy()
grupo_loc = aulas_geral[['Grupo', 'Jardim','Satélite','Vicentina']].copy()

In [8]:
Professores = {'Name':['Allan','Carlos','Henrique','Yasmin','Malu','Tanze','Okamoto','Edu','Wolf','Odin','Renan','Gabiraes','Nicolas','Novaes','Jorge','Kaori']}
profes = pd.DataFrame(Professores)
profes.shape

(16, 1)

In [9]:
# sortear uma aula para cada professor
# usar a função de minimizar para completar os horarios e turmas restantes

In [10]:
aulas_segunda = aulas_geral[aulas_geral['segunda'] == 1].copy()

In [11]:
aulas_segunda

,Grupo,Horário,Unidade,Dias da Semana,segunda,terça,quarta,quinta,07:00:00,08:00:00,...,17:00:00,18:00:00,18:10:00,18:30:00,19:00:00,20:00:00,20:30:00,Jardim,Satélite,Vicentina
0,MBA DAVI BRAGA,14:00:00,Vicentina,2ª,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,MBA IDEIAS EM AÇÃO,19:00:00,Vicentina,2ª,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,SAVANAH PRESENCIAL,07:00:00,Vicentina,"2ª , 3ª , 4ª , 5ª",1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
5,FERNANDO CAJE - PRESENCIAL,08:00:00,Vicentina,"2ª , 4ª",1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
7,KNOXVILLE PRESENCIAL,08:00:00,Vicentina,"2ª , 3ª , 4ª , 5ª",1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,1
8,GREAT BRITAIN PRESENCIAL,09:00:00,Vicentina,"2ª , 3ª , 4ª , 5ª",1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
9,GARLAND PRESENCIAL,18:00:00,Vicentina,"2ª , 3ª , 4ª , 5ª",1,1,1,1,0,0,...,0,1,0,0,0,0,0,0,0,1
10,TAMPA PRESENCIAL,19:00:00,Vicentina,"2ª , 3ª , 4ª , 5ª",1,1,1,1,0,0,...,0,0,0,0,1,0,0,0,0,1
11,CAPE COD PRESENCIAL,20:00:00,Vicentina,"2ª , 4ª , 5ª",1,0,1,1,0,0,...,0,0,0,0,0,1,0,0,0,1
12,ERILENE - PRESENCIAL/ONLINE,08:00:00,Jardim,2ª DOUBLE,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [11]:
all_teacher = profes['Name'].to_list()
all_group = aulas_segunda['Grupo'].to_list()

In [11]:
all_teacher = profes['Name'].to_list()
all_group = aulas_segunda['Grupo'].to_list()

alocacoes = {}
model = cp_model.CpModel()
for i in all_teacher:
    for j in all_group:
        h = aulas_segunda.loc[aulas_segunda['Grupo'] == j, 'Horário'].values[0]
        alocacoes[(i,j,h)] = model.NewBoolVar(f"{i}_converinglesson_{j}_period_{h}")

for a in all_group:
    b = aulas_segunda.loc[aulas_segunda['Grupo'] == a, 'Horário'].values[0]
    model.Add(sum(alocacoes[(c,a,b)] for c in all_teacher) == 1)

for professor in all_teacher:
    for horario in aulas_segunda['Horário'].unique():
        model.Add(sum(alocacoes[(professor, row['Grupo'], horario)] for _, row in aulas_segunda.iterrows() if row['Horário'] == horario) <= 1)



In [17]:
model = cp_model.CpModel()

# Dicionário para armazenar as variáveis booleanas
alocacoes = {}

# Criar variáveis booleanas para as combinações de turma e horário existentes
for professor in all_teacher:
    for _, row in aulas_segunda.iterrows():
        turma = row['Grupo']
        horario = row['Horário']
        
        nome_var = f'{professor}_{turma}_{horario}'
        alocacoes[(professor, turma, horario)] = model.NewBoolVar(nome_var)

# Adicionar a restrição: um professor não pode estar em duas turmas no mesmo horário
for _, row in aulas_segunda.iterrows():
    turma = row['Grupo']
    horario = row['Horário']
    model.Add(sum(alocacoes[(professor, turma, horario)] for professor in all_teacher) == 1)

# Nova restrição: um professor só pode estar em uma turma por horário
for professor in all_teacher:
    for horario in aulas_segunda['Horário'].unique():
        model.Add(sum(alocacoes[(professor, row['Grupo'], horario)] for _, row in aulas_segunda.iterrows() if row['Horário'] == horario) <= 1)


# Criar o solver e rodar o modelo

solver = cp_model.CpSolver()
status = solver.Solve(model)
# Imprimir as alocações
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!")
    
    # Imprimir as alocações
    for professor in all_teacher:
        for _, row in aulas_segunda.iterrows():
            turma = row['Grupo']
            horario = row['Horário']# Nova restrição: um professor só pode estar em uma turma por horário
            # Verificar se o professor foi alocado para esta turma no horário específico
            if solver.BooleanValue(alocacoes[(professor, turma, horario)]):
                print(f"{professor} está alocado para {turma} no horário {horario}")
else:
    print("Nenhuma solução viável foi encontrada.")

Solução encontrada!
Allan está alocado para MBA IDEIAS EM AÇÃO no horário 19:00:00
Allan está alocado para SAVANAH PRESENCIAL no horário 07:00:00
Allan está alocado para FERNANDO CAJE - PRESENCIAL no horário 08:00:00
Allan está alocado para GARLAND PRESENCIAL no horário 18:00:00
Allan está alocado para FAIRFAX PRESENCIAL no horário 16:00:00
Allan está alocado para DELTA PRESENCIAL no horário 12:00:00
Allan está alocado para FALLS CHURCH PRESENCIAL no horário 20:30:00
Allan está alocado para DERBY PRESENCIAL no horário 09:00:00
Allan está alocado para SEATTLE PRESENCIAL no horário 15:00:00
Allan está alocado para LANCASTER PRESENCIAL no horário 17:00:00
Carlos está alocado para KNOXVILLE PRESENCIAL no horário 08:00:00
Carlos está alocado para GREAT BRITAIN PRESENCIAL no horário 09:00:00
Carlos está alocado para TAMPA PRESENCIAL no horário 19:00:00
Carlos está alocado para CAPE COD PRESENCIAL no horário 20:00:00
Carlos está alocado para WARRINGTON PRESENCIAL no horário 18:00:00
Carlos es